In [19]:
import os
import h5py

import socket
import struct
import pickle
import sys

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

from threading import Thread
from threading import Lock


import time

from tqdm import tqdm

import copy

In [20]:
rounds = 5
local_epoch = 10
users = 2 # number of clients

In [21]:
generator = tf.keras.models.load_model('gan_model_gen')
discriminator = tf.keras.models.load_model('gan_model_dis')

/home/romulo/code/Federated-Learning-and-Split-Learning-with-raspberry-pi/env/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


/home/romulo/code/Federated-Learning-and-Split-Learning-with-raspberry-pi/env/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [22]:
# # Função para plotar imagens geradas
# def plot_generated_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1)):
#     noise = np.random.normal(0, 1, size=[examples, 100])
#     generated_images = generator.predict(noise)
#     generated_images = generated_images.reshape(examples, 28, 28)

#     plt.figure(figsize=figsize)
#     for i in range(generated_images.shape[0]):
#         plt.subplot(dim[0], dim[1], i + 1)
#         plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
#         plt.axis('off')
#     plt.tight_layout()
#     plt.savefig(f'gan_generated_image_epoch_{epoch}.png')

In [23]:
# plot_generated_images(10, generator)

In [24]:
# # Função para fazer previsões (discriminar)
# def predict_images(discriminator, images):
#     # Normalizar as imagens para o intervalo [-1, 1]
#     images = (images - 127.5) / 127.5
    
#     # Achatando as imagens para vetores
#     images_flat = images.reshape(images.shape[0], -1)
    
#     # Fazer previsões
#     predictions = discriminator.predict(images_flat)
    
#     return predictions

# # Carregar imagens reais do MNIST para teste (ou substituir por suas próprias imagens)
# (train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()

# # Selecionar um subconjunto de imagens para teste
# num_images_to_test = 10
# test_images = train_images[:num_images_to_test]

# # Adapte as dimensões das imagens e adicione a dimensão do canal
# test_images = test_images.reshape(num_images_to_test, 28, 28, 1).astype('float32')

# # Fazer previsões usando o discriminador
# predictions = predict_images(discriminator, test_images)

# # Exibir as imagens e suas previsões
# plt.figure(figsize=(26, 4))
# for i in range(num_images_to_test):
#     plt.subplot(2, num_images_to_test, i + 1)
#     plt.imshow(test_images[i, :, :, 0], cmap='gray')
#     plt.title("Real")
#     plt.axis('off')

#     plt.subplot(2, num_images_to_test, i + 1 + num_images_to_test)
#     plt.bar(range(1), predictions[i, 0], tick_label=['Fake'])
#     plt.title("Discriminator Prediction")
#     plt.ylim([0, 1])

# plt.show()


## variables

In [25]:
clientsoclist = [0]*users

start_time = 0
weight_count = 0

global_weights = copy.deepcopy(generator.get_weights())

datasetsize = [0]*users
weights_list = [0]*users

lock = Lock()

## Comunication overhead

In [26]:
total_sendsize_list = []
total_receivesize_list = []

client_sendsize_list = [[] for i in range(users)]
client_receivesize_list = [[] for i in range(users)]

train_sendsize_list = [] 
train_receivesize_list = []

### Required socket functions

In [27]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    l_send = len(msg)
    msg = struct.pack('>I', l_send) + msg
    sock.sendall(msg)
    return l_send

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg, msglen

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [28]:
def average_weights(w, datasize):
    """
    Returns the average of the weights.
    """
    
    # Scale each weight by its corresponding data size
    for i, data in enumerate(datasize):
        for j in range(len(w[i])):
            w[i][j] = tf.multiply(w[i][j], float(data))
    
    # Create a deep copy of the first set of weights
    w_avg = copy.deepcopy(w[0])

    # Sum the scaled weights for each layer
    for j in range(len(w_avg)):
        for i in range(1, len(w)):
            w_avg[j] = tf.add(w_avg[j], w[i][j])
        
        # Calculate the average by dividing the sum by the total data size
        w_avg[j] = tf.divide(w_avg[j], float(sum(datasize)))

    return w_avg


## Receive users before training

In [29]:
def run_thread(func, num_user):
    global clientsoclist
    global start_time
    
    thrs = []
    for i in range(num_user):
        conn, addr = s.accept()
        print('Conntected with', addr)
        # append client socket on list
        clientsoclist[i] = conn
        args = (i, num_user, conn)
        thread = Thread(target=func, args=args)
        thrs.append(thread)
        thread.start()
    print("timmer start!")
    start_time = time.time()    # store start time
    for thread in thrs:
        thread.join()
    end_time = time.time()  # store end time
    print("TrainingTime: {} sec".format(end_time - start_time))

In [30]:
def receive(userid, num_users, conn): #thread for receive clients
    global weight_count
    
    global datasetsize


    msg = {
        'rounds': rounds,
        'client_id': userid,
        'local_epoch': local_epoch
    }

    datasize = send_msg(conn, msg)    #send epoch
    total_sendsize_list.append(datasize)
    client_sendsize_list[userid].append(datasize)

    train_dataset_size, datasize = recv_msg(conn)    # get total_batch of train dataset
    total_receivesize_list.append(datasize)
    client_receivesize_list[userid].append(datasize)
    
    
    with lock:
        datasetsize[userid] = train_dataset_size
        weight_count += 1
    
    train(userid, train_dataset_size, num_users, conn)

In [31]:
def train(userid, train_dataset_size, num_users, client_conn):
    global weights_list
    global global_weights
    global weight_count
    global val_acc
    
    for r in range(rounds):
        with lock:
            if weight_count == num_users:
                for i, conn in enumerate(clientsoclist):
                    datasize = send_msg(conn, global_weights)
                    total_sendsize_list.append(datasize)
                    client_sendsize_list[i].append(datasize)
                    train_sendsize_list.append(datasize)
                    weight_count = 0

        client_weights, datasize = recv_msg(client_conn)
        total_receivesize_list.append(datasize)
        client_receivesize_list[userid].append(datasize)
        train_receivesize_list.append(datasize)

        weights_list[userid] = client_weights
        print("User" + str(userid) + "'s Round " + str(r + 1) +  " is done")
        with lock:
            weight_count += 1
            if weight_count == num_users:
                #average
                global_weights = average_weights(weights_list, datasetsize)
                #train_discriminator(epochs, batch_size)
                
        
    

In [ ]:
# Função para treinar apenas o Discriminador da GAN
def train_discriminator(epochs=1, batch_size=128):
    batch_count = x_train.shape[0] // batch_size

    for e in range(epochs):
        for _ in range(batch_count):
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
            noise = np.random.normal(0, 1, size=[batch_size, latent_dim])
            generated_images = generator.predict(noise)

            X = np.concatenate([image_batch, generated_images])
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9  # Rótulos suavizados para o treinamento estável

            # Treina o discriminador
            d_loss = discriminator.train_on_batch(X, y_dis)

        print(f'Época {e+1}/{epochs}, Discriminador Loss: {d_loss}')

In [32]:
host = socket.gethostbyname(socket.gethostname())
port = 10080
print(host)

127.0.1.1


In [33]:
s = socket.socket()
s.bind((host, port))
s.listen(5)

In [34]:
run_thread(receive, users)

Conntected with ('127.0.0.1', 57266)
Conntected with ('127.0.0.1', 60172)
timmer start!
User1's Round 1 is done
User0's Round 1 is done
User1's Round 2 is done
User0's Round 2 is done
User1's Round 3 is done
User0's Round 3 is done
User1's Round 4 is done
User0's Round 4 is done
User1's Round 5 is done
User0's Round 5 is done
TrainingTime: 315.41452407836914 sec


In [35]:
end_time = time.time()  # store end time
print("TrainingTime: {} sec".format(end_time - start_time))

TrainingTime: 315.43290734291077 sec


## Print all of communication overhead

In [36]:
# print('val_acc list')
# for acc in val_acc:
#     print(acc)

print('\n')
print('---total_sendsize_list---')
total_size = 0
for size in total_sendsize_list:
#     print(size)
    total_size += size
print("total_sendsize size: {} bytes".format(total_size))
print('\n')

print('---total_receivesize_list---')
total_size = 0
for size in total_receivesize_list:
#     print(size)
    total_size += size
print("total receive sizes: {} bytes".format(total_size) )
print('\n')

for i in range(users):
    print('---client_sendsize_list(user{})---'.format(i))
    total_size = 0
    for size in client_sendsize_list[i]:
#         print(size)
        total_size += size
    print("total client_sendsizes(user{}): {} bytes".format(i, total_size))
    print('\n')

    print('---client_receivesize_list(user{})---'.format(i))
    total_size = 0
    for size in client_receivesize_list[i]:
#         print(size)
        total_size += size
    print("total client_receive sizes(user{}): {} bytes".format(i, total_size))
    print('\n')

print('---train_sendsize_list---')
total_size = 0
for size in train_sendsize_list:
#     print(size)
    total_size += size
print("total train_sendsizes: {} bytes".format(total_size))
print('\n')

print('---train_receivesize_list---')
total_size = 0
for size in train_receivesize_list:
#     print(size)
    total_size += size
print("total train_receivesizes: {} bytes".format(total_size))
print('\n')




---total_sendsize_list---
total_sendsize size: 59750786 bytes


---total_receivesize_list---
total receive sizes: 59749310 bytes


---client_sendsize_list(user0)---
total client_sendsizes(user0): 29875393 bytes


---client_receivesize_list(user0)---
total client_receive sizes(user0): 29874655 bytes


---client_sendsize_list(user1)---
total client_sendsizes(user1): 29875393 bytes


---client_receivesize_list(user1)---
total client_receive sizes(user1): 29874655 bytes


---train_sendsize_list---
total train_sendsizes: 59750672 bytes


---train_receivesize_list---
total train_receivesizes: 59749280 bytes


